In [1]:
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.preprocessing import OrdinalEncoder
import matplotlib.pyplot as plt
from utils import *

In [79]:
import pickle
df_all_cols = pd.DataFrame()

use_config = 'with_interaction_10'

for benchmark in ["TabArena", "TabZilla", 'Grinsztajn']:
    with open(f"../../ftd_variants_{benchmark}.pkl", "rb") as f:
        results = pickle.load(f)
    results.keys()

    for dataset_name in results.keys():
        try:
            if 'dataset' in df_all_cols.columns and dataset_name in df_all_cols.dataset.unique():
                continue
            sig = pd.DataFrame(results[dataset_name][use_config]['csv']['significances']).transpose()

            csv_detected = pd.Series(results[dataset_name][use_config]['csv']['dtypes'], name='csv_detected')
            true = pd.Series(results[dataset_name][use_config]['openml']['orig_dtypes'], name='true')

            res_df = pd.merge(csv_detected.loc[true.index], true, left_index=True, right_index=True)
            res_df["dataset"] = dataset_name
            res_df['benchmark'] = benchmark
            res_df = pd.merge(res_df, sig, left_index=True, right_index=True, how='left')

            df_all_cols = pd.concat([df_all_cols, res_df], axis=0)
        except:
            print(f"Error processing dataset {dataset_name} in benchmark {benchmark}")
df_all_cols.shape

Error processing dataset guillermo in benchmark TabZilla


(9268, 29)

In [80]:
df_all_cols

,csv_detected,true,dataset,benchmark,test_irrelevant_mean,test_combination_test_2_superior,test_combination_test_4_superior,test_combination_test_8_superior,test_combination_test_16_superior,test_linear_superior,...,"test_arithmetic-combination[2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096]_superior_single-best",test_arithmetic-best_superior_single-best,test_combination_test_32_superior,test_combination_test_64_superior,test_combination_test_128_superior,test_combination_test_256_superior,test_combination_test_512_superior,test_combination_test_1024_superior,test_combination_test_2048_superior,test_combination_test_4096_superior
frequency,numeric,numeric,airfoil_self_noise,TabArena,0.03125,0.96875,0.84375,0.21875,0.03125,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
attack-angle,categorical,categorical,airfoil_self_noise,TabArena,0.03125,1.00000,1.00000,1.00000,1.00000,1.00000,...,0.96875,0.06250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
chord-length,numeric,numeric,airfoil_self_noise,TabArena,0.03125,1.00000,0.78125,NaN,NaN,0.96875,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
free-stream-velocity,low-cardinality,numeric,airfoil_self_noise,TabArena,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
suction-side-displacement-thickness,numeric,numeric,airfoil_self_noise,TabArena,0.03125,1.00000,1.00000,0.96875,0.96875,1.00000,...,NaN,NaN,1.00000,1.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
entropy_Valence,numeric,numeric,superconduct,Grinsztajn,0.03125,1.00000,1.00000,1.00000,1.00000,1.00000,...,0.03125,0.03125,1.00000,1.0,1.00000,1.00000,NaN,NaN,NaN,NaN
wtd_entropy_Valence,numeric,numeric,superconduct,Grinsztajn,0.03125,1.00000,0.06250,0.03125,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
wtd_range_Valence,numeric,numeric,superconduct,Grinsztajn,0.03125,1.00000,1.00000,1.00000,0.90625,NaN,...,NaN,NaN,0.03125,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std_Valence,numeric,numeric,superconduct,Grinsztajn,0.03125,1.00000,1.00000,1.00000,1.00000,1.00000,...,0.03125,0.03125,1.00000,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [82]:
print(f"Binary columns: {sum(df_all_cols['true']=='binary')}")
print(f"Low-cardinality columns: {sum(df_all_cols['csv_detected']=='low-cardinality')}")
print(f"Unpredictive Mean columns: {sum(df_all_cols['csv_detected']=='irrelevant')}")
print(f"Unpredictive mean top-5: {df_all_cols[df_all_cols['csv_detected']=='irrelevant']['dataset'].value_counts().head(5)}")
print(f"Unpredictive mean num/cat: {df_all_cols[df_all_cols['csv_detected']=='irrelevant']['true'].value_counts(normalize=True)}")
# df_current[np.logical_and(df_current['csv_detected']=='irrelevant', df_current['true']=='categorical')]
# df_tmp = df_current[np.logical_and(df_current['csv_detected']=='irrelevant', df_current['true']=='numeric')]
# df_tmp.loc[df_tmp['dataset']!='guillermo']['test_irrelevant_mean'].hist()


Binary columns: 5220
Low-cardinality columns: 766
Unpredictive Mean columns: 1192
Unpredictive mean top-5: dataset
Bioresponse                        275
topo_2_1                           248
kddcup09_appetency                 124
taiwanese_bankruptcy_prediction     54
jannis                              44
Name: count, dtype: int64
Unpredictive mean num/cat: true
numeric        0.965604
categorical    0.034396
Name: proportion, dtype: float64


In [83]:
df_current = df_all_cols.loc[df_all_cols['csv_detected']!='binary']
df_current = df_current.loc[df_current['csv_detected']!='low-cardinality']
df_current = df_current.loc[df_current['csv_detected']!='irrelevant']
df_current


,csv_detected,true,dataset,benchmark,test_irrelevant_mean,test_combination_test_2_superior,test_combination_test_4_superior,test_combination_test_8_superior,test_combination_test_16_superior,test_linear_superior,...,"test_arithmetic-combination[2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096]_superior_single-best",test_arithmetic-best_superior_single-best,test_combination_test_32_superior,test_combination_test_64_superior,test_combination_test_128_superior,test_combination_test_256_superior,test_combination_test_512_superior,test_combination_test_1024_superior,test_combination_test_2048_superior,test_combination_test_4096_superior
frequency,numeric,numeric,airfoil_self_noise,TabArena,0.03125,0.96875,0.84375,0.21875,0.03125,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
attack-angle,categorical,categorical,airfoil_self_noise,TabArena,0.03125,1.00000,1.00000,1.00000,1.00000,1.00000,...,0.96875,0.06250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
chord-length,numeric,numeric,airfoil_self_noise,TabArena,0.03125,1.00000,0.78125,NaN,NaN,0.96875,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
suction-side-displacement-thickness,numeric,numeric,airfoil_self_noise,TabArena,0.03125,1.00000,1.00000,0.96875,0.96875,1.00000,...,NaN,NaN,1.00000,1.0,NaN,NaN,NaN,NaN,NaN,NaN
RESOURCE,categorical,categorical,Amazon_employee_access,TabArena,0.03125,1.00000,1.00000,1.00000,1.00000,1.00000,...,1.00000,1.00000,1.00000,1.0,1.00000,0.96875,0.68750,1.00000,0.96875,0.9375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
entropy_Valence,numeric,numeric,superconduct,Grinsztajn,0.03125,1.00000,1.00000,1.00000,1.00000,1.00000,...,0.03125,0.03125,1.00000,1.0,1.00000,1.00000,NaN,NaN,NaN,NaN
wtd_entropy_Valence,numeric,numeric,superconduct,Grinsztajn,0.03125,1.00000,0.06250,0.03125,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
wtd_range_Valence,numeric,numeric,superconduct,Grinsztajn,0.03125,1.00000,1.00000,1.00000,0.90625,NaN,...,NaN,NaN,0.03125,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std_Valence,numeric,numeric,superconduct,Grinsztajn,0.03125,1.00000,1.00000,1.00000,1.00000,1.00000,...,0.03125,0.03125,1.00000,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
# Mismatch what we detect vs. whats true
print(pd.concat([df_current['true'].value_counts(),df_current['csv_detected'].value_counts()],axis=1))

             count  count
numeric       1856   1785
categorical    154    225


In [85]:
comb_cols = [f'test_combination_test_{i}_superior' for i in [2,4,8,16, 32, 64, 128, 256, 512, 1024, 2048]]
print(f"{df_current.shape[0]} columns in total")
for bins in [2,4,8,16, 32, 64, 128, 256, 512, 1024, 2048]:
    print(f"No combination test for {bins} bins: {df_current[f'test_combination_test_{bins}_superior'].isna().sum()}")
for bins in [2,4,8,16, 32, 64, 128, 256, 512, 1024, 2048]:
    print(f"Significant combination test for {bins} bins: {(df_current[f'test_combination_test_{bins}_superior'].dropna()<0.05).sum()}")

summarized_comb = df_current[comb_cols].min(axis=1)
print(f"Filtered through combination test: {sum(summarized_comb<0.05)}")
print(df_current[summarized_comb<0.05]['true'].value_counts(normalize=True))

# df_current[summarized_comb.isna()]

2010 columns in total
No combination test for 2 bins: 693
No combination test for 4 bins: 1015
No combination test for 8 bins: 1234
No combination test for 16 bins: 1405
No combination test for 32 bins: 1524
No combination test for 64 bins: 1631
No combination test for 128 bins: 1711
No combination test for 256 bins: 1757
No combination test for 512 bins: 1803
No combination test for 1024 bins: 1835
No combination test for 2048 bins: 1863
Significant combination test for 2 bins: 322
Significant combination test for 4 bins: 202
Significant combination test for 8 bins: 110
Significant combination test for 16 bins: 64
Significant combination test for 32 bins: 43
Significant combination test for 64 bins: 25
Significant combination test for 128 bins: 11
Significant combination test for 256 bins: 11
Significant combination test for 512 bins: 3
Significant combination test for 1024 bins: 3
Significant combination test for 2048 bins: 0
Filtered through combination test: 794
true
numeric       

In [86]:
df_current = df_current.loc[~(summarized_comb<0.05)]
df_current

,csv_detected,true,dataset,benchmark,test_irrelevant_mean,test_combination_test_2_superior,test_combination_test_4_superior,test_combination_test_8_superior,test_combination_test_16_superior,test_linear_superior,...,"test_arithmetic-combination[2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096]_superior_single-best",test_arithmetic-best_superior_single-best,test_combination_test_32_superior,test_combination_test_64_superior,test_combination_test_128_superior,test_combination_test_256_superior,test_combination_test_512_superior,test_combination_test_1024_superior,test_combination_test_2048_superior,test_combination_test_4096_superior
attack-angle,categorical,categorical,airfoil_self_noise,TabArena,0.03125,1.0,1.00000,1.00000,1.00000,1.00000,...,0.96875,0.06250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
chord-length,numeric,numeric,airfoil_self_noise,TabArena,0.03125,1.0,0.78125,NaN,NaN,0.96875,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
suction-side-displacement-thickness,numeric,numeric,airfoil_self_noise,TabArena,0.03125,1.0,1.00000,0.96875,0.96875,1.00000,...,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
RESOURCE,categorical,categorical,Amazon_employee_access,TabArena,0.03125,1.0,1.00000,1.00000,1.00000,1.00000,...,1.00000,1.00000,1.0,1.0,1.00000,0.96875,0.68750,1.00000,0.96875,0.93750
MGR_ID,categorical,categorical,Amazon_employee_access,TabArena,0.03125,1.0,1.00000,1.00000,1.00000,1.00000,...,1.00000,0.03125,1.0,1.0,1.00000,1.00000,1.00000,1.00000,0.96875,0.96875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mean_Valence,numeric,numeric,superconduct,Grinsztajn,0.03125,1.0,1.00000,1.00000,1.00000,1.00000,...,0.03125,0.03125,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
gmean_Valence,numeric,numeric,superconduct,Grinsztajn,0.03125,1.0,1.00000,1.00000,1.00000,1.00000,...,0.03125,0.03125,1.0,1.0,1.00000,NaN,NaN,NaN,NaN,NaN
entropy_Valence,numeric,numeric,superconduct,Grinsztajn,0.03125,1.0,1.00000,1.00000,1.00000,1.00000,...,0.03125,0.03125,1.0,1.0,1.00000,1.00000,NaN,NaN,NaN,NaN
std_Valence,numeric,numeric,superconduct,Grinsztajn,0.03125,1.0,1.00000,1.00000,1.00000,1.00000,...,0.03125,0.03125,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
# Interpolation test with match criterion
print(f"Filtered columns: {sum(df_current['test_mean_superior_to_linear']>0.05)}")
print(df_current[df_current['test_mean_superior_to_linear']>0.05]['true'].value_counts())
df_current[np.logical_and(df_current['test_mean_superior_to_linear']>0.05, df_current['true']=='categorical')]

Filtered columns: 148
true
numeric        140
categorical      8
Name: count, dtype: int64


,csv_detected,true,dataset,benchmark,test_irrelevant_mean,test_combination_test_2_superior,test_combination_test_4_superior,test_combination_test_8_superior,test_combination_test_16_superior,test_linear_superior,...,"test_arithmetic-combination[2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096]_superior_single-best",test_arithmetic-best_superior_single-best,test_combination_test_32_superior,test_combination_test_64_superior,test_combination_test_128_superior,test_combination_test_256_superior,test_combination_test_512_superior,test_combination_test_1024_superior,test_combination_test_2048_superior,test_combination_test_4096_superior
EaseofOnlinebooking,numeric,categorical,customer_satisfaction_in_airline,TabArena,0.03125,1.0000,1.00000,NaN,NaN,1.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Onboardservice,numeric,categorical,customer_satisfaction_in_airline,TabArena,0.03125,1.0000,1.00000,NaN,NaN,1.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Checkinservice,numeric,categorical,customer_satisfaction_in_airline,TabArena,0.03125,1.0000,1.00000,NaN,NaN,1.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
STENOK_AN,numeric,categorical,MIC,TabArena,0.03125,0.0625,0.06250,NaN,NaN,0.21875,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TIME_B_S,numeric,categorical,MIC,TabArena,0.03125,0.9375,0.78125,0.31250,NaN,0.84375,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mucous_membranes,numeric,categorical,colic,TabZilla,0.03125,0.9375,0.59375,NaN,NaN,0.31250,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
V55,numeric,categorical,albert,TabZilla,0.03125,1.0000,0.93750,1.00000,1.0,0.84375,...,NaN,NaN,0.90625,0.9375,0.9375,0.96875,0.78125,0.96875,0.96875,0.6875
cat103,numeric,categorical,Allstate_Claims_Severity,Grinsztajn,0.03125,1.0000,1.00000,0.96875,NaN,0.96875,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
# Interpolation test with match criterion
print(f"Filtered columns: {sum(df_current['test_mean_superior_to_poly2']>0.05)}")
print(df_current[df_current['test_mean_superior_to_poly2']>0.05]['true'].value_counts())
df_current[np.logical_and(df_current['test_mean_superior_to_poly2']>0.05, df_current['true']=='categorical')]

Filtered columns: 50
true
numeric        46
categorical     4
Name: count, dtype: int64


,csv_detected,true,dataset,benchmark,test_irrelevant_mean,test_combination_test_2_superior,test_combination_test_4_superior,test_combination_test_8_superior,test_combination_test_16_superior,test_linear_superior,...,"test_arithmetic-combination[2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096]_superior_single-best",test_arithmetic-best_superior_single-best,test_combination_test_32_superior,test_combination_test_64_superior,test_combination_test_128_superior,test_combination_test_256_superior,test_combination_test_512_superior,test_combination_test_1024_superior,test_combination_test_2048_superior,test_combination_test_4096_superior
state,numeric,categorical,churn,TabArena,0.03125,1.0,0.93750,0.96875,1.0,1.0,...,NaN,NaN,0.9375,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Foodanddrink,numeric,categorical,customer_satisfaction_in_airline,TabArena,0.03125,1.0,1.00000,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cat111,numeric,categorical,Allstate_Claims_Severity,Grinsztajn,0.03125,1.0,0.96875,0.78125,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
weekday,numeric,categorical,delays_zurich_transport,Grinsztajn,0.03125,1.0,0.90625,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [89]:
# Interpolation test with match criterion
print(f"Filtered columns: {sum(df_current['test_mean_superior_to_poly3']>0.05)}")
print(df_current[df_current['test_mean_superior_to_poly3']>0.05]['true'].value_counts())
df_current[np.logical_and(df_current['test_mean_superior_to_poly3']>0.05, df_current['true']=='categorical')]

Filtered columns: 33
true
numeric        27
categorical     6
Name: count, dtype: int64


,csv_detected,true,dataset,benchmark,test_irrelevant_mean,test_combination_test_2_superior,test_combination_test_4_superior,test_combination_test_8_superior,test_combination_test_16_superior,test_linear_superior,...,"test_arithmetic-combination[2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096]_superior_single-best",test_arithmetic-best_superior_single-best,test_combination_test_32_superior,test_combination_test_64_superior,test_combination_test_128_superior,test_combination_test_256_superior,test_combination_test_512_superior,test_combination_test_1024_superior,test_combination_test_2048_superior,test_combination_test_4096_superior
Inflightwifiservice,numeric,categorical,customer_satisfaction_in_airline,TabArena,0.03125,1.0,1.0,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Inflightentertainment,numeric,categorical,customer_satisfaction_in_airline,TabArena,0.03125,1.0,1.0,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cleanliness,numeric,categorical,customer_satisfaction_in_airline,TabArena,0.03125,1.0,1.0,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
site_of_lesion,numeric,categorical,colic,TabZilla,0.03125,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,0.84375,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cat91,numeric,categorical,Allstate_Claims_Severity,Grinsztajn,0.03125,1.0,1.0,1.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cat101,numeric,categorical,Allstate_Claims_Severity,Grinsztajn,0.03125,1.0,1.0,1.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
# Interpolation test with win criterion
print(f"Filtered columns: {sum(df_current['test_linear_superior']<0.05)}")
print(df_current[df_current['test_linear_superior']<0.05]['true'].value_counts())
df_current[np.logical_and(df_current['test_linear_superior']<0.05, df_current['true']=='categorical')]

Filtered columns: 12
true
numeric    12
Name: count, dtype: int64


,csv_detected,true,dataset,benchmark,test_irrelevant_mean,test_combination_test_2_superior,test_combination_test_4_superior,test_combination_test_8_superior,test_combination_test_16_superior,test_linear_superior,...,"test_arithmetic-combination[2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096]_superior_single-best",test_arithmetic-best_superior_single-best,test_combination_test_32_superior,test_combination_test_64_superior,test_combination_test_128_superior,test_combination_test_256_superior,test_combination_test_512_superior,test_combination_test_1024_superior,test_combination_test_2048_superior,test_combination_test_4096_superior


In [91]:
# Interpolation test with win criterion
print(f"Filtered columns: {sum(df_current['test_poly2_superior']<0.05)}")
print(df_current[df_current['test_poly2_superior']<0.05]['true'].value_counts())
df_current[np.logical_and(df_current['test_poly2_superior']<0.05, df_current['true']=='categorical')]

Filtered columns: 1
true
numeric    1
Name: count, dtype: int64


,csv_detected,true,dataset,benchmark,test_irrelevant_mean,test_combination_test_2_superior,test_combination_test_4_superior,test_combination_test_8_superior,test_combination_test_16_superior,test_linear_superior,...,"test_arithmetic-combination[2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096]_superior_single-best",test_arithmetic-best_superior_single-best,test_combination_test_32_superior,test_combination_test_64_superior,test_combination_test_128_superior,test_combination_test_256_superior,test_combination_test_512_superior,test_combination_test_1024_superior,test_combination_test_2048_superior,test_combination_test_4096_superior


In [92]:
# Interpolation test with win criterion
print(f"Filtered columns: {sum(df_current['test_poly3_superior']<0.05)}")
print(df_current[df_current['test_poly3_superior']<0.05]['true'].value_counts())
df_current[np.logical_and(df_current['test_poly3_superior']<0.05, df_current['true']=='categorical')]

Filtered columns: 1
true
categorical    1
Name: count, dtype: int64


,csv_detected,true,dataset,benchmark,test_irrelevant_mean,test_combination_test_2_superior,test_combination_test_4_superior,test_combination_test_8_superior,test_combination_test_16_superior,test_linear_superior,...,"test_arithmetic-combination[2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096]_superior_single-best",test_arithmetic-best_superior_single-best,test_combination_test_32_superior,test_combination_test_64_superior,test_combination_test_128_superior,test_combination_test_256_superior,test_combination_test_512_superior,test_combination_test_1024_superior,test_combination_test_2048_superior,test_combination_test_4096_superior
cat91,numeric,categorical,Allstate_Claims_Severity,Grinsztajn,0.03125,1.0,1.0,1.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [93]:
df_current = df_current.loc[df_current[['test_mean_superior_to_linear','test_mean_superior_to_poly2','test_mean_superior_to_poly3']].max(axis=1)<=0.05]

In [94]:
df_current['true'].value_counts()

true
numeric        257
categorical     33
Name: count, dtype: int64

In [95]:
df_current[df_current['true']=='numeric'].dataset.value_counts()

dataset
superconductivity                             56
superconduct                                  49
Bioresponse                                   29
nomao                                         29
SpeedDating                                   10
APSFailure                                     9
road-safety                                    6
concrete_compressive_strength                  5
students_dropout_and_academic_success          5
physiochemical_protein                         4
maternal_health_risk                           4
eye_movements                                  4
artificial-characters                          4
airlines                                       3
Bike_Sharing_Demand                            3
house_sales                                    3
albert                                         2
jungle_chess_2pcs_raw_endgame_complete         2
seattlecrime6                                  2
anneal                                         2
Food_Deliver

In [96]:
df_current[df_current['true']=='categorical']

,csv_detected,true,dataset,benchmark,test_irrelevant_mean,test_combination_test_2_superior,test_combination_test_4_superior,test_combination_test_8_superior,test_combination_test_16_superior,test_linear_superior,...,"test_arithmetic-combination[2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096]_superior_single-best",test_arithmetic-best_superior_single-best,test_combination_test_32_superior,test_combination_test_64_superior,test_combination_test_128_superior,test_combination_test_256_superior,test_combination_test_512_superior,test_combination_test_1024_superior,test_combination_test_2048_superior,test_combination_test_4096_superior
attack-angle,categorical,categorical,airfoil_self_noise,TabArena,0.03125,1.0000,1.0000,1.00000,1.0,1.0,...,0.96875,0.06250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
RESOURCE,categorical,categorical,Amazon_employee_access,TabArena,0.03125,1.0000,1.0000,1.00000,1.0,1.0,...,1.00000,1.00000,1.0,1.0,1.0,0.96875,0.6875,1.0,0.96875,0.93750
MGR_ID,categorical,categorical,Amazon_employee_access,TabArena,0.03125,1.0000,1.0000,1.00000,1.0,1.0,...,1.00000,0.03125,1.0,1.0,1.0,1.00000,1.0000,1.0,0.96875,0.96875
ROLE_ROLLUP_2,categorical,categorical,Amazon_employee_access,TabArena,0.03125,1.0000,1.0000,1.00000,1.0,1.0,...,0.03125,0.03125,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN
ROLE_DEPTNAME,categorical,categorical,Amazon_employee_access,TabArena,0.03125,1.0000,1.0000,1.00000,1.0,1.0,...,0.15625,0.03125,1.0,1.0,1.0,1.00000,NaN,NaN,NaN,NaN
ROLE_TITLE,categorical,categorical,Amazon_employee_access,TabArena,0.03125,1.0000,1.0000,1.00000,1.0,1.0,...,0.03125,0.03125,1.0,1.0,1.0,1.00000,NaN,NaN,NaN,NaN
ROLE_FAMILY_DESC,categorical,categorical,Amazon_employee_access,TabArena,0.03125,1.0000,1.0000,1.00000,1.0,1.0,...,0.03125,0.03125,1.0,1.0,1.0,1.00000,1.0000,1.0,1.00000,NaN
ROLE_FAMILY,categorical,categorical,Amazon_employee_access,TabArena,0.03125,1.0000,1.0000,1.00000,1.0,1.0,...,0.68750,0.03125,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
ROLE_CODE,categorical,categorical,Amazon_employee_access,TabArena,0.03125,1.0000,1.0000,1.00000,1.0,1.0,...,0.03125,0.03125,1.0,1.0,1.0,1.00000,NaN,NaN,NaN,NaN
Seatcomfort,categorical,categorical,customer_satisfaction_in_airline,TabArena,0.03125,1.0000,1.0000,NaN,NaN,1.0,...,1.00000,0.03125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Interaction test

In [98]:
df_current[df_current['csv_detected']=='numeric']['true'].value_counts()

true
numeric        135
categorical     13
Name: count, dtype: int64

In [99]:
df_current[df_current['csv_detected']=='numeric'].sort_values(['true']).head(20)

,csv_detected,true,dataset,benchmark,test_irrelevant_mean,test_combination_test_2_superior,test_combination_test_4_superior,test_combination_test_8_superior,test_combination_test_16_superior,test_linear_superior,...,"test_arithmetic-combination[2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096]_superior_single-best",test_arithmetic-best_superior_single-best,test_combination_test_32_superior,test_combination_test_64_superior,test_combination_test_128_superior,test_combination_test_256_superior,test_combination_test_512_superior,test_combination_test_1024_superior,test_combination_test_2048_superior,test_combination_test_4096_superior
V24,numeric,categorical,albert,TabZilla,0.03125,1.0,1.0,1.0,1.0000,1.0,...,0.03125,0.03125,1.00000,1.00000,1.0,1.0,1.0,1.0,1.0,1.0
V26,numeric,categorical,albert,TabZilla,0.03125,1.0,1.0,1.0,1.0000,1.0,...,0.03125,0.03125,1.00000,1.00000,1.0,1.0,1.0,1.0,1.0,NaN
V25,numeric,categorical,albert,TabZilla,0.03125,1.0,1.0,1.0,1.0000,1.0,...,0.03125,0.03125,1.00000,1.00000,1.0,1.0,1.0,1.0,1.0,1.0
V37,numeric,categorical,albert,TabZilla,0.03125,1.0,1.0,1.0,1.0000,1.0,...,0.03125,0.03125,1.00000,1.00000,1.0,1.0,1.0,1.0,1.0,1.0
fiber_ID,numeric,categorical,SDSS17,TabArena,0.03125,1.0,1.0,1.0,1.0000,1.0,...,0.03125,0.03125,1.00000,1.00000,1.0,1.0,1.0,NaN,NaN,NaN
V23,numeric,categorical,albert,TabZilla,0.03125,1.0,1.0,1.0,1.0000,1.0,...,0.03125,0.03125,1.00000,1.00000,1.0,1.0,1.0,1.0,1.0,1.0
V17,numeric,categorical,albert,TabZilla,0.03125,1.0,1.0,1.0,1.0000,1.0,...,0.03125,0.03125,1.00000,1.00000,1.0,1.0,1.0,1.0,1.0,1.0
V16,numeric,categorical,albert,TabZilla,0.03125,1.0,1.0,1.0,1.0000,1.0,...,0.03125,0.03125,1.00000,1.00000,1.0,1.0,1.0,1.0,1.0,1.0
V15,numeric,categorical,albert,TabZilla,0.03125,1.0,1.0,1.0,1.0000,1.0,...,0.03125,0.03125,1.00000,1.00000,1.0,1.0,1.0,NaN,NaN,NaN
V39,numeric,categorical,albert,TabZilla,0.03125,1.0,1.0,1.0,1.0000,1.0,...,0.03125,0.03125,1.00000,1.00000,1.0,1.0,1.0,1.0,1.0,1.0


In [100]:
df_current = df_current[df_current['csv_detected']=='categorical']
df_current

,csv_detected,true,dataset,benchmark,test_irrelevant_mean,test_combination_test_2_superior,test_combination_test_4_superior,test_combination_test_8_superior,test_combination_test_16_superior,test_linear_superior,...,"test_arithmetic-combination[2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096]_superior_single-best",test_arithmetic-best_superior_single-best,test_combination_test_32_superior,test_combination_test_64_superior,test_combination_test_128_superior,test_combination_test_256_superior,test_combination_test_512_superior,test_combination_test_1024_superior,test_combination_test_2048_superior,test_combination_test_4096_superior
attack-angle,categorical,categorical,airfoil_self_noise,TabArena,0.03125,1.0,1.0,1.0,1.0,1.0,...,0.96875,0.06250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
RESOURCE,categorical,categorical,Amazon_employee_access,TabArena,0.03125,1.0,1.0,1.0,1.0,1.0,...,1.00000,1.00000,1.0,1.0,1.0,0.96875,0.6875,1.0,0.96875,0.93750
MGR_ID,categorical,categorical,Amazon_employee_access,TabArena,0.03125,1.0,1.0,1.0,1.0,1.0,...,1.00000,0.03125,1.0,1.0,1.0,1.00000,1.0000,1.0,0.96875,0.96875
ROLE_ROLLUP_2,categorical,categorical,Amazon_employee_access,TabArena,0.03125,1.0,1.0,1.0,1.0,1.0,...,0.03125,0.03125,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN
ROLE_DEPTNAME,categorical,categorical,Amazon_employee_access,TabArena,0.03125,1.0,1.0,1.0,1.0,1.0,...,0.15625,0.03125,1.0,1.0,1.0,1.00000,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
std_ElectronAffinity,categorical,numeric,superconduct,Grinsztajn,0.03125,1.0,1.0,1.0,1.0,1.0,...,1.00000,1.00000,1.0,1.0,1.0,1.00000,1.0000,1.0,NaN,NaN
mean_FusionHeat,categorical,numeric,superconduct,Grinsztajn,0.03125,1.0,1.0,1.0,1.0,1.0,...,0.31250,0.31250,1.0,1.0,1.0,1.00000,1.0000,1.0,1.00000,NaN
mean_ThermalConductivity,categorical,numeric,superconduct,Grinsztajn,0.03125,1.0,1.0,1.0,1.0,1.0,...,1.00000,1.00000,1.0,1.0,1.0,1.00000,1.0000,1.0,NaN,NaN
gmean_ThermalConductivity,categorical,numeric,superconduct,Grinsztajn,0.03125,1.0,1.0,1.0,1.0,1.0,...,1.00000,1.00000,1.0,1.0,1.0,1.00000,1.0000,1.0,1.00000,NaN


In [101]:
df_current['true'].value_counts()

true
numeric        122
categorical     20
Name: count, dtype: int64

In [102]:
df_current[df_current['true']=='numeric'].dataset.value_counts()

dataset
Bioresponse                                   29
superconductivity                             27
superconduct                                  13
SpeedDating                                    9
APSFailure                                     6
artificial-characters                          4
eye_movements                                  4
maternal_health_risk                           4
road-safety                                    3
airlines                                       3
seattlecrime6                                  2
anneal                                         2
vehicle                                        2
concrete_compressive_strength                  2
electricity                                    1
Bank_Customer_Churn                            1
MIC                                            1
jungle_chess_2pcs_raw_endgame_complete         1
HR_Analytics_Job_Change_of_Data_Scientists     1
analcatdata_supreme                            1
particulate-

In [103]:
df_current[df_current['true']=='categorical'].dataset.value_counts()

dataset
Amazon_employee_access                 8
customer_satisfaction_in_airline       3
albert                                 3
Allstate_Claims_Severity               3
airfoil_self_noise                     1
Mercedes_Benz_Greener_Manufacturing    1
particulate-matter-ukair-2017          1
Name: count, dtype: int64